In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score,KFold
import statsmodels.formula.api as smf
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier
from sklearn.metrics import roc_auc_score

C:\Users\hp\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = pd.read_csv('D:\Traina.csv')
test = pd.read_csv('D:\Testa.csv')

In [3]:
train.shape, test.shape

((1763, 1559), (756, 1558))

In [6]:
train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_1550,feature_1551,feature_1552,feature_1553,feature_1554,feature_1555,feature_1556,feature_1557,feature_1558,Class
0,100,160,1.6000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,83,4.1500,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,99,150,1.5151,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,40,1.0000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,234,19.5000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train['Class'].value_counts()

0    1620
1     143
Name: Class, dtype: int64

In [4]:
sc = StandardScaler()

In [5]:
train1 = train.drop(['Class'],axis =1)
y = train['Class']
trains1 = sc.fit_transform(train1)
tests1 = sc.transform(test)



In [8]:
LR=LogisticRegression(max_iter = 800)

kfold = KFold(shuffle=True,n_splits=10,random_state=0)
lr_results = cross_val_score(LR, trains1, y,cv=kfold, scoring='roc_auc')
print(np.mean(lr_results))
print(np.std(lr_results,ddof=1))
  

0.8109858445844254
0.05394363540311609


In [9]:
model_lr = LR.fit(trains1,y)
ypred_lr = model_lr.predict(tests1)

In [10]:
y_lr = pd.DataFrame(ypred_lr)
y_lr.to_excel("D:/anomalies1.xlsx",index=False)

In [9]:
rf=RandomForestClassifier(n_estimators = 400, max_depth = 6,criterion = 'entropy')

kfold = KFold(shuffle=True,n_splits=10,random_state=0)
rf_results = cross_val_score(rf, trains1, y,cv=kfold, scoring='roc_auc')
print(np.mean(rf_results))
print(np.std(rf_results,ddof=1))

0.9182964402769217
0.03476221628214517


In [10]:
model_rf = rf.fit(trains1,y)
ypred_rf = model_rf.predict(tests1)

In [15]:
y_rf = pd.DataFrame(ypred_rf)
y_rf.to_excel("D:/anomalies2.xlsx",index=False)

In [18]:
y_rf[0].value_counts()

0    743
1     13
Name: 0, dtype: int64

In [6]:
pval=[]
selected_cols=[]
discarded_cols=[]
A0=train[train['Class']==0]
A1=train[train['Class']==1]

from scipy.stats import f_oneway
for col in train1.columns:
    res=f_oneway(A0[col],A1[col])
    pval.append(res[1])
    if res[1]<0.05:
        selected_cols.append(col)     #means are significantly different
    else:
        discarded_cols.append(col)    # Null Hypothesis is True 
print(discarded_cols)
len(discarded_cols)

C:\Users\hp\Anaconda3\lib\site-packages\scipy\stats\stats.py:3235: RuntimeWarning: invalid value encountered in double_scalars
  f = msb / msw


['feature_1', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_11', 'feature_13', 'feature_14', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_67', 'feature_68', 'feature_69', 'feature_72', 'feature_73', 'feature_74', 'feature_75', 'feature_76', 'feature_77', 'feature_78', 'feature_79', 'feature_80', 'feature_81', 'feature_82', 'feature_83', 'feature_84', 'feature_85', 'feature_86', 'feature_87', 'feature_88', 'feature_89'

1188

In [7]:
len(selected_cols)

370

In [8]:
X_update=train[selected_cols]
X_update.shape

(1763, 370)

In [9]:
test_update = test[selected_cols]
test_update.shape

(756, 370)

In [10]:
X_std = sc.fit_transform(X_update)
tests_update  = sc.transform(test_update)

In [39]:
rf=RandomForestClassifier(n_estimators = 400, max_depth = 6,criterion = 'entropy')

kfold = KFold(shuffle=True,n_splits=10,random_state=0)
rf_results2 = cross_val_score(rf, X_std, y,cv=kfold, scoring='roc_auc')
print(np.mean(rf_results2))
print(np.std(rf_results2,ddof=1))

0.899966097183379
0.05062034020425024


In [40]:
model_rf1 = rf.fit(X_std,y)
ypred_rf1 = model_rf1.predict(tests_update)

In [28]:
ypred_rf1

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_update,y,test_size=0.3,random_state=7)

In [17]:
X_trains = sc.fit_transform(X_train)
X_tests = sc.transform(X_test)

In [18]:
from lightgbm import LGBMClassifier
lgb_fit_params={"early_stopping_rounds":500, 
            
            "eval_set" : [(X_tests,y_test)],
            'eval_names': ['valid'],
            'verbose':100
           }
lgb_params = {'boosting_type': 'gbdt',
 
 'verbose': 0,
 'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'lambda_l1': 0.01,
 'lambda_l2': 0.01,
 'learning_rate': 0.01,
 'max_bin': 255,
 'max_depth': 6,
 'min_data_in_bin': 1,
 'min_data_in_leaf': 1,
 'num_leaves': 31}

In [19]:
lgb = LGBMClassifier(n_estimators=20000, **lgb_params, random_state=123456789, n_jobs=-1)
lgb.fit(X_trains, y_train, **lgb_fit_params)
lgb.best_iteration_

Training until validation scores don't improve for 500 rounds
[100]	valid's binary_logloss: 0.16318
[200]	valid's binary_logloss: 0.160755
[300]	valid's binary_logloss: 0.165054
[400]	valid's binary_logloss: 0.171078
[500]	valid's binary_logloss: 0.176101
[600]	valid's binary_logloss: 0.182219
Early stopping, best iteration is:
[153]	valid's binary_logloss: 0.160141


153

In [20]:
clf_lgb=LGBMClassifier(n_estimators=int(lgb.best_iteration_*2.3), **lgb_params)
lgb_model=clf_lgb.fit(X_std, y)

In [21]:
kfold = KFold(shuffle=True,n_splits=10,random_state=0)
lgb_score=cross_val_score(X=X_std,y=y,estimator=clf_lgb,scoring='roc_auc',cv=kfold)

In [22]:
np.mean(lgb_score)

0.8993961680870409

In [23]:
ypred_lgb = lgb_model.predict(tests_update)

In [24]:
y_lgb =  pd.DataFrame(ypred_lgb)


In [25]:
y_lgb[0].value_counts()

0    705
1     51
Name: 0, dtype: int64

In [74]:
ypred_f = (ypred_lgb * 0.6) + (ypred_xgb * 0.3) + (ypred_rf1 * 0.1)

In [75]:
y_f = pd.DataFrame(ypred_f)

In [76]:
y_f[0].value_counts()

0.0    665
0.6     40
0.9     33
1.0      7
0.3      7
0.4      4
Name: 0, dtype: int64

In [77]:
y_f.to_excel("D:/anomalies4.xlsx",index=False)

In [78]:
ypred_f1 = (ypred_lgb * 0.5) + (ypred_xgb * 0.4) + (ypred_rf1 * 0.1) 

In [79]:
y_f1 = pd.DataFrame(ypred_f1)

In [80]:
y_f1[0].value_counts()

0.0    665
0.5     44
0.9     33
0.4      7
1.0      7
Name: 0, dtype: int64

In [81]:
y_f1.to_excel("D:/anomalies5.xlsx",index=False)

In [89]:
ypred_f2 = (ypred_lgb * 0.4) + (ypred_xgb * 0.5) + (ypred_ann * 0.1) 

In [90]:
y_f2 = pd.DataFrame(ypred_f2)

In [91]:
y_f2[0].value_counts()

0.0    655
0.5     31
1.0     30
0.8     14
0.3     10
0.7     10
0.2      6
Name: 0, dtype: int64

In [92]:
y_f2.to_excel("D:/anomalies6.xlsx",index=False)

In [72]:
ypred_f3 = (ypred_lgb * 0.45) + (ypred_xgb * 0.35) + (ypred_ann2 * 0.2)

In [73]:
y_f3 = pd.DataFrame(ypred_f3)

In [74]:
y_f3[0].value_counts()

0.00    653
1.00     28
0.45     23
0.65     19
0.80     11
0.20     10
0.35      7
0.55      5
Name: 0, dtype: int64

In [75]:
y_f3.to_excel("D:/anomalies7.xlsx",index=False)

In [76]:
ypred_f4 = (ypred_lgb * 0.5) + (ypred_xgb * 0.4) + (ypred_ann2 * 0.1)

In [77]:
y_f4 = pd.DataFrame(ypred_f4)
y_f4[0].value_counts()

0.0    653
0.5     28
1.0     28
0.6     19
0.9     11
0.1     10
0.4      7
Name: 0, dtype: int64

In [85]:
ypred_f5 = (ypred_lgb * 0.6) + (ypred_xgb * 0.3) + (ypred_ann1 * 0.1)

In [86]:
y_f5 = pd.DataFrame(ypred_f5)
y_f5[0].value_counts()

0.0    662
0.6     42
0.9     39
0.3     12
0.1      1
Name: 0, dtype: int64

In [78]:
y_f4.to_excel("D:/anomalies8.xlsx",index=False)

In [53]:
ypred_f6 = (ypred_lgb * 0.5) + (ypred_xgb * 0.5)  

In [54]:
y_f6 = pd.DataFrame(ypred_f6)
y_f6[0].value_counts()

0.0    700
1.0     41
0.5     15
Name: 0, dtype: int64

In [55]:
y_f6.to_excel("D:/anomalies10.xlsx",index=False)

In [56]:
from xgboost import XGBClassifier
xgb=XGBClassifier(
 learning_rate =0.01,
 n_estimators=1200,
 max_depth=6,
 colsample_bytree=0.8,
 seed=100,n_jobs = -1)
xgb_model=xgb.fit(X_std,y)

In [57]:

xgb_score=cross_val_score(X=X_std,y=y,estimator=xgb,scoring='roc_auc',cv=5)

In [58]:
np.mean(xgb_score)

0.9015922428997142

In [59]:
ypred_xgb = xgb_model.predict(tests_update)
y_xgb = pd.DataFrame(ypred_xgb)


In [60]:
y_xgb[0].value_counts()

0    702
1     54
Name: 0, dtype: int64

In [11]:
grd = GradientBoostingClassifier(learning_rate =0.01,n_estimators=6500,max_depth=6)
grd_model=grd.fit(X_std,y)

In [12]:
grd_score=cross_val_score(X=X_std,y=y,estimator=grd,scoring='roc_auc',cv=5)
np.mean(grd_score)

KeyboardInterrupt: 

In [67]:
clf_lgb1=LGBMClassifier(n_estimators=int(lgb.best_iteration_*2.0), **lgb_params)
lgb_model1=clf_lgb1.fit(X_trains, y_train)

In [68]:
kfold = KFold(shuffle=True,n_splits=10,random_state=0)
lgb_score1=cross_val_score(X=X_trains,y=y_train,estimator=clf_lgb1,scoring='roc_auc',cv=kfold)

In [69]:
np.mean(lgb_score1)

0.8940882201187572

In [70]:
ypred_lgb1 = lgb_model1.predict(tests_update)

In [71]:
y_lgb1 =  pd.DataFrame(ypred_lgb1)
y_lgb1[0].value_counts()

0    676
1     80
Name: 0, dtype: int64

In [31]:
from sklearn.neural_network import MLPClassifier

In [57]:
ANN=MLPClassifier(hidden_layer_sizes=(50,80,60))

In [58]:
kfold = KFold(shuffle=True,n_splits=10,random_state=0)
ann_score=cross_val_score(X=X_std,y=y,estimator=ANN,scoring='roc_auc',cv=kfold)

In [59]:
np.mean(ann_score)

0.6457495734656252

In [55]:
ANN.fit(X_std,y)
ypred_ann=ANN.predict(tests_update)

In [56]:
y_ann =  pd.DataFrame(ypred_ann)
y_ann[0].value_counts()

0    690
1     66
Name: 0, dtype: int64

In [60]:
ann2 = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)

In [61]:
kfold = KFold(shuffle=True,n_splits=10,random_state=0)
ann_score2=cross_val_score(X=X_std,y=y,estimator=ann2,scoring='roc_auc',cv=kfold)

In [62]:
np.mean(ann_score2)

0.6735811645003908

In [63]:
ann2.fit(X_std,y)
ypred_ann2=ann2.predict(tests_update)

In [64]:
y_ann2 =  pd.DataFrame(ypred_ann2)
y_ann2[0].value_counts()

0    694
1     62
Name: 0, dtype: int64

In [32]:
ann1 = MLPClassifier(solver='adam', activation='relu',alpha=1e-4,hidden_layer_sizes=(50,50,50), random_state=42,max_iter=400,learning_rate_init=.1)



In [33]:
kfold = KFold(shuffle=True,n_splits=10,random_state=0)
ann_score1=cross_val_score(X=X_std ,y=y,estimator=ann1,scoring='roc_auc',cv=kfold)

In [34]:
np.mean(ann_score1)

0.79206875008505

In [35]:
ann1.fit(X_std,y)
ypred_ann1=ann1.predict(tests_update)

In [51]:
y_ann1 =  pd.DataFrame(ypred_ann1)
y_ann1[0].value_counts()

0    755
1      1
Name: 0, dtype: int64